In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
timer.start()

notifica = Notifica()
# notifica.download_todas_notificacoes()
notifica.read_todas_notificacoes()
notifica.save()
# notifica.load()
print(timer.ftime())
notifica.shape()

00:00:06.219


(2296606, 1055528, 10936, 1227993, 2149)

In [3]:
timer.start()
casos_confirmados = CasosConfirmados()
# casos_confirmados.read()
# casos_confirmados.save()
casos_confirmados.load()
print(timer.ftime())
casos_confirmados.shape()

00:02:34.666


(581195, 10614, 425720, 141444)

In [4]:
casosn = notifica.get_casos()
casosn['duplicado'] = 0
casosn['manter'] = 0
casosc = casos_confirmados.get_casos()
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,556082
CASO DESCARTADO,782634
CASO SUSPEITO,950230
IGNORADO,648
SINDROME GRIPAL NAO ESPECIFICADA,7012


In [5]:
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 522056
209922 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,14798
CASO DESCARTADO,127851
CASO SUSPEITO,168009
IGNORADO,49
SINDROME GRIPAL NAO ESPECIFICADA,1427


In [6]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [7]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: 83607
35003 pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,3040
CASO DESCARTADO,17662
CASO SUSPEITO,27701
IGNORADO,7
SINDROME GRIPAL NAO ESPECIFICADA,194


In [8]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [9]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo data_nascimento: 20218
9982 pacientes pelo data_nascimento que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,883
CASO DESCARTADO,2706
CASO SUSPEITO,6608
IGNORADO,9
SINDROME GRIPAL NAO ESPECIFICADA,30


In [10]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [11]:
casos_duplicados = casosn.loc[(casosn['hash_resid'].notnull()) & (casosn.duplicated('hash_resid',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_resid'))} pacientes pelo hash_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_resid: 2660
1302 pacientes pelo hash_resid que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,62
CASO DESCARTADO,241
CASO SUSPEITO,1054
SINDROME GRIPAL NAO ESPECIFICADA,1


In [12]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [13]:
casos_duplicados = casosn.loc[(casosn['hash_atend'].notnull()) & (casosn.duplicated('hash_atend',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_atend'))} pacientes pelo hash_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_atend: 385
192 pacientes pelo hash_atend que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,16
CASO DESCARTADO,31
CASO SUSPEITO,144
IGNORADO,2


In [14]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [15]:
casos_duplicados = casosn.loc[(casosn['hash_diag'].notnull()) & (casosn.duplicated('hash_diag',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_diag: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_diag'))} pacientes pelo hash_diag que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_diag'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_hash_diag.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_diag: 6847
3295 pacientes pelo hash_diag que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,499
CASO DESCARTADO,1303
CASO SUSPEITO,1737
IGNORADO,2
SINDROME GRIPAL NAO ESPECIFICADA,11


In [18]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [30]:
notifica.save(casosn)

In [19]:
print(f"Das {notifica.shape()[0]} notificações baixadas, {notifica.shape()[0] - casosn.shape[0]} ({len(all_casos_duplicados)}) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de {casosn.shape[0]}")
allcasosn = notifica.get_casos()
allcasosn.loc[all_casos_duplicados,'id'].to_csv(join(output,'casos_duplicados_removidos.csv'), index=False)
allcasosn.loc[all_duplicados_mantidos,'id'].to_csv(join(output,'casos_duplicados_mantidos.csv'), index=False)

Das 2296606 notificações baixadas, 376077 (376077) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de 1920529


In [20]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.to_excel(join(output,'casos_confirmados_nao_notifica.xlsx'),index=False)
casos_confirmados_nao_notifica.shape

(30519, 23)

In [21]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.to_excel(join(output,'casos_confirmados_notifica.xlsx'),index=False)
casos_confirmados_notifica.shape

(543769, 55)

In [22]:
notifica_nao_casos_confirmados = casosn.loc[set(casosn.index.tolist()) - set(idx_casos_notifica)]

In [23]:
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_classificacao_final'] == 2]
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_status_notificacao'].isin([1,2])]

notifica_nao_casos_confirmados['month'] = notifica_nao_casos_confirmados.apply(lambda x: x['data_notificacao'].month, axis=1)
nnccm = notifica_nao_casos_confirmados.groupby(by='month')[['id']].count()
nnccm.index = [ meses[int(x)-1] for x in nnccm.index] 
nnccm

,id
jan,543
fev,1881
mar,39
abril,105
mai,138
jun,436
jul,492
ago,375
set,196
out,147


In [24]:
notifica_nao_casos_confirmados.to_excel(join(output,'notifica_nao_casos_confirmados.xlsx'), index=False)
notifica_nao_casos_confirmados.shape

(4999, 56)

In [25]:
casosn.loc[casosn['hash_diag']=='WESLEIZOLET05022021']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado,mantido
1485572,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,1,CURA,2021-01-28,2021-02-10,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,2,ENCERRADA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-08,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,0,0


In [26]:
casosn.loc[casosn['hash_resid']=='WESLEIZOLET24TOLEDO']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado,mantido
1485572,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,1,CURA,2021-01-28,2021-02-10,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,2,ENCERRADA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-08,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,0,0


In [27]:
casosn.loc[casosn['hash_atend']=='WESLEIZOLET24TOLEDO']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado,mantido
1485572,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,1,CURA,2021-01-28,2021-02-10,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,2,ENCERRADA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-08,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,0,0


In [28]:
casosn.loc[casosn['hash_mae']=='WESLEIZOLETMARIADELOURDESDASILVA']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado,mantido
1485572,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,1,CURA,2021-01-28,2021-02-10,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,2,ENCERRADA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-08,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,0,0


In [29]:
casosn.loc[casosn['hash_nasc']=='WESLEIZOLET14121996']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado,mantido
1485572,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,1,CURA,2021-01-28,2021-02-10,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,2,ENCERRADA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-08,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,0,0
